In [1]:
import re
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
from opencc import OpenCC

### 讀取資料

In [2]:
def load_data(crime):
    tree_leaves = []
    name = []
    node = []
    with open('./data/' + crime +'.pkl', 'rb') as file:
        while 1:
            try:
                tree = pickle.load(file)
                names = pickle.load(file)
                name.append(names)
                for i in tree:
                    tree_leaves.append(i.leaves())
                    for j in i:
                        node.append(j)
            except:
                #print("讀取完畢")
                break
    file.close()
    return name, node, tree_leaves

### 建立姓名列表，翻成簡體字

In [3]:
def simplified_Word(t):
    to_convert= str(t)
    cc = OpenCC('t2s')  # (Optional )convert from Simplified Chinese to Traditional Chinese
    content = cc.convert(to_convert)#一樣轉成簡體字，這樣Stanford的系統才會比較準確
    return content

def list_of_name(name):  
    name_list = []
    for j in range(0,len(name)):
        temp = []
        for i in name[j]:
            temp.append(simplified_Word(i[0]))
        name_list.append(temp)
    return name_list

### 定位姓名、犯罪名稱位置，取出句子

In [4]:
def locate(tree, crime, name_list):
    cri = simplified_Word(crime)
    lo = [] #犯罪名稱位置
    na = [] #姓名位置
    me = [] #相對姓名
    string = ''
    sentence = [] #相應句子
    new = []
    new_tree = []
    for i in range(0,len(tree)):
        temp = []
        temp2 = []
        temp3 = []
        for item in enumerate(tree[i].leaves()):
            if cri in item[1]:
                temp.append(item[0])
                #print(item[1])
                #print(item[0])
                #print(h)
                #print('------')
            for n in name_list[i]:
                if n in item[1]:
                    temp2.append(item[0])
                    temp3.append(item[1])
                    #print(item[1])
                    #print(item[0])
                    #print(h)
                    #print('------')
                    #h = h + 1
                #else:
                #    temp.append(0)
                #    temp2.append(0)
                #    temp3.append("無")
        if (len(temp) > 0) & (len(temp2) > 0) & (len(temp3) > 0):
            na.append(temp2)
            me.append(temp3)   
            lo.append(temp)
            #for o in tree[i]:
            #    new.append(o)
            #new_tree.append(new)
            new_tree.append(tree[i])
    #sentence.append(string.join(tree[i].leaves()))
    return lo, na, me, new_tree

### 文法樹詞性

In [5]:
def tag_POS(tree):
    POS_tag = []
    tree = tree.pos()
    for item in tree:
        word,POS = str(item).replace('(','').replace(')','').replace("'",'').replace(" ",'').split(',')
        POS_tag.append([word,POS])
    return POS_tag

# 實際操作

In [6]:
#for cr in crime_name:
#讀取資料
cr = "洗錢"
name, node, tree_leaves = load_data("洗錢_2")
#建立姓名列表
name_list = list_of_name(name)
#定位姓名、犯罪名稱在樹上的位置
lo, na, me, new_tree = locate(node, cr, name_list)

In [7]:
tag = []
for i in range(0,len(new_tree)):
    tag.append(tag_POS(new_tree[i]))
#tag

In [8]:
for_a = ['洗钱']
for_b = ['涉', '承认', '赌', '违反', '认定', '前科', '捕', '嫌', '逮', '起诉']
for_b_verb = ['犯']
for_c = ['律师', '翻摄', '金管会']
y = 0
j = 0
string = " "
for i in range(0,len(new_tree)):
    a = 0
    b = 0
    c = 0
    for w, p in tag[i]:
        #print(w,p)
        for item in for_a:
            if (item in w):
                a = a + 1
            #print(a)
            #print("good")
        for item in for_b:
            if (item in w):
                b = b + 1
        for item in for_b_verb:
            if (item in w) & (p == "VV"):
                b = b + 1
        for item in for_c:
            if (item in w):
                c = c + 1
    #print(a)
    #print(b)
    if (a > 0) & (b > 0) & (c == 0):
        string = '' #相應句子
        print('\033[1;34m'+ string.join(new_tree[i].leaves()) + '\033[0m')
        print("----")
        #print('\033[1;34m' + str(y) + '\033[0m')
        #y = y + 1
    else:
        print(string.join(new_tree[i].leaves()))
        print("----")
        #print(j)
        #j = j + 1
    
    

数月 前 ， 高市 刑警大队 责由 经济 组 加强 查缉 ， 在 专案 人员 追查 下 ， 查知 有 一 绰号 「 林仔 」 为首 的 诈欺 集团 ， 扮演 厦门 的 诈欺 集团 在 台 洗钱 等 工作 ， 成员 分布 花莲 、 高雄 、 嘉义 等 地 ， 报请 高雄 检察署 检察官 林永富 指挥 侦办 。
----
台北地方法院审理时，李宏泰表示自己因「查厂」情事，担心安危，故在未告知源兴公司情况下离开中国，但并未侵占公司资产，他反控源兴公司违反中国法令，以兰光公司为白手套，利用购买发票等方式套汇洗钱，且本案中的十二张汇票并未遭兑领，未有侵占事实。
----
桃园县站主任吴天仁调任第二处专门委员，中机组主任王汝浩调任廉政处专门委员，嘉义市站主任张祥山调任缉毒中心专门委员，苗栗县站主任侯杰调任洗钱防制中心专门委员，台南县站主任江桂馨调任经济犯罪防制中心专门委员兼副主任。
----
兴票案肇因八十八年十二月间，国民党与华夏投资公司因认宋楚瑜担任国民党秘书长期间，涉嫌侵吞、挪用巨额党产，而向北检递状控告宋楚瑜及其机要秘书杨云黛、小姨子陈碧云等人涉犯侵占、洗钱、税捐稽征法等多项罪嫌。
----
〔记者张协升╱台中报导〕台湾樱花公司董事长张宗玺八十七年间涉嫌以假买卖虚增公司业绩，并以假土地买卖掏空樱花建设公司八千余万元，遭台中地检署以背信、洗钱防制法等罪嫌起诉，羁押四个多月后，台中地院十一日开庭，认为张某已无串证逃亡之虞，谕令以四千万元交保候传，其家人十二日筹款缴交后，张某已获交保，创下中部地区交保的最高金额纪录。
----
张宗玺去年九月八日遭台中地检署传讯后声押获准，隔月被依背信、洗钱防制法等罪嫌起诉，求处六年徒刑，法院审理阶段，张宗玺续遭羁押。
----
日本方面，去年五月二十一日，我与日本合作，查获日本史上最大规模黑帮洗钱案通缉犯山根敬，瓦解黑帮组织山口组资金来源；去年六月二日，与菲国警方合作，在国内查获山口组重要成员根本晃跨国走私枪械案。
----
国泰世华副总饶世湛昨天表示，涉案的台北分行前行员吕嘉惠还没有离职前，银行内部风险控管机制已经发现资金进出有可疑情形，立即展开调查，由于事件重大，涉及行外体系资金动向的调查，另依洗钱防制法规范处理，主动通知检调单位进行侦办。
----
办案人员随即在国泰世华银行主管的配合下，反向追出离职不久的台北分行临柜行员吕嘉惠涉有重


# 結果
---
### 預測成1: 86 ; 預測成0: 59
#### confusion_matrix
[13, 2]
<br>
[46, 84]
<br><br>
### 準確率:
0: 0.22
<br>
1: 0.98

In [ ]:
"""
是1測成1 84
是0測成1 2
是0測成0: 13 
是1測成0: 46
"""